In [50]:
import pandas as pd
import sqlite3

# Exercise 03 : Aggregations

## create a connection to the database using the library sqlite3

In [51]:
con = sqlite3.connect('data/checking-logs.sqlite')
con

## get the schema of the table test

In [52]:
schema_df = pd.read_sql('PRAGMA table_info(test);', con)
schema_df

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [53]:
test_df = pd.read_sql('SELECT * FROM test LIMIT 10;', con)
test_df

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45,2020-04-18 10:56:55
1,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:05:48
2,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:06:13
3,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:06:35
4,user_17,project1,2020-04-18 07:56:45,2020-04-21 19:04:25
5,user_17,project1,2020-04-18 07:56:45,2020-04-26 18:11:53
6,user_17,project1,2020-04-18 07:56:45,2020-05-02 13:21:34
7,user_17,project1,2020-04-18 07:56:45,2020-05-11 10:55:04
8,user_17,project1,2020-04-18 07:56:45,2020-05-11 11:12:26
9,user_17,project1,2020-04-18 07:56:45,2020-05-12 19:31:22


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
- do this by joining the table with the table deadlines
- the difference should be displayed in hours
- do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier
- the value should be stored in the dataframe df_min with the corresponding uid

In [54]:
join_table = """
SELECT
    test.uid,
    test.labname,
    MIN((deadlines.deadlines - strftime('%s', test.first_commit_ts)) / 3600.0) AS min_delta_hours
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY 
    labname
"""

df_min = pd.read_sql(join_table, con)

df_min

,uid,labname,min_delta_hours
0,user_18,laba04,4.196667
1,user_25,laba04s,2.867500
2,user_18,laba05,10.973611
3,user_17,laba06,80.641111
4,user_17,laba06s,78.345000


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [55]:
join_table = """
SELECT
    test.uid,
    test.labname,
    MAX((deadlines.deadlines - strftime('%s', test.first_commit_ts)) / 3600.0) AS min_delta_hours
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY 
    labname
"""

df_max = pd.read_sql(join_table, con)

df_max

,uid,labname,min_delta_hours
0,user_30,laba04,202.385000
1,user_30,laba04s,201.139444
2,user_14,laba05,84.448611
3,user_3,laba06,182.055278
4,user_3,laba06s,135.285833


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [56]:
join_table = """
SELECT
    test.labname,
    AVG((deadlines.deadlines - strftime('%s', test.first_commit_ts)) / 3600.0) AS min_delta_hours
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY 
    labname
"""

df_avg = pd.read_sql(join_table, con)

df_avg

,labname,min_delta_hours
0,laba04,107.435562
1,laba04s,62.251557
2,laba05,58.890157
3,laba06,161.115625
4,laba06s,114.743537


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference
- using only one query, create a table with the columns: uid, avg_diff, pageviews
- uid is the uids that exist in the test
- avg_diff is the average delta between the first commit and the lab deadline per user
- pageviews is the number of Newsfeed visits per user
- do not take the lab ’project1’ into account
- store it to the dataframe views_diff
- use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [96]:
query = """
CREATE TABLE IF NOT EXISTS views_diff AS
SELECT
    test.uid,
    AVG((deadlines.deadlines - strftime('%s', test.first_commit_ts)) / 3600.0) AS avg_diff,
    COUNT(test.first_view_ts) AS pageviews
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY 
    uid, labname
"""
con.execute(query)
con.commit()


In [97]:
views_diff = pd.read_sql("SELECT * FROM views_diff", con)
views_diff

,uid,avg_diff,pageviews
0,user_1,6.894722,28
1,user_1,6.796667,28
2,user_1,28.744722,28
3,user_1,175.556667,28
4,user_1,107.606111,28
5,user_10,39.585278,89
6,user_10,39.368056,89
7,user_10,52.542500,89
8,user_10,132.341944,89
9,user_10,112.374444,89


In [99]:
float(views_diff["avg_diff"].corr(views_diff["pageviews"]))

0.1487844724876787

## close the connection

In [100]:
con.close()